In [1]:
# Neural Networks

# Training Procedure

## 1. Define the Neural Networks that has some learnable parameters ( or weights).
## 2. Iterate over the Dataset of inputs.
## 3. Process Input through the network.
## 4. Compute the loss (How far is the output from being correct).
## 5. Propagate gradients back into the Network's Parameters
## 6. Update the weights of the network, typically using a simple update rule: WEIGHT = WEIGHT - LEARNING_RATE * GRADIENT

In [2]:
# Import statemnts
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as fxnl
import torch.optim as opt


In [3]:
#### Define the Neural Network

class  NeuralNet(nn.Module):

    ## Constructor(s)
    def __init__(self):
        super(NeuralNet, self).__init__()

        # 2D Convolution Layer with 1 input channel, 6 output channels and a convolution kernel size(3*3)
        self.conv_layer_one = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3) 

        # 2D Convolution Layer with (conv_layer_one.out_channel) input channels, 16 output channels and kernel size (3*3)
        self.conv_layer_two = nn.Conv2d(in_channels= 6, out_channels= 16, kernel_size=3)

        self.fc_one = nn.Linear(in_features=16 * 6 * 6, out_features=120)
        self.fc_two = nn.Linear(in_features=120, out_features=84)
        self.fc_three = nn.Linear(in_features=84, out_features=10)
    
    # Forwarding Method
    def forward(self, forward_linker):
        #Max Pooling over a (2, 2) window
        forward_linker = fxnl.max_pool2d(fxnl.relu(self.conv_layer_one(forward_linker)), (2, 2))

        forward_linker = fxnl.max_pool2d(fxnl.relu(self.conv_layer_two(forward_linker)), 2)
        forward_linker = forward_linker.view(-1, self.num_flat_features(forward_linker))

        forward_linker = fxnl.relu(self.fc_one(forward_linker))
        forward_linker = fxnl.relu(self.fc_two(forward_linker))
        forward_linker = self.fc_three(forward_linker)

        return forward_linker

    # Helper Method(s)
    def num_flat_features(self, linker):
        size = linker.size()[1:]
        num_features = 1

        for val in size:
            num_features *= val
        
        return num_features



In [4]:
neural_net = NeuralNet()
print(neural_net)

NeuralNet(
  (conv_layer_one): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer_two): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc_one): Linear(in_features=576, out_features=120, bias=True)
  (fc_two): Linear(in_features=120, out_features=84, bias=True)
  (fc_three): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
parameters = list(neural_net.parameters())
print(len(parameters))
for id in range(len(parameters)):
    print(parameters[id].size())

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [6]:
# Random input 
rand_input = torch.randn(1, 1, 32, 32)
print(rand_input)

tensor([[[[-0.4921,  1.5270,  0.1252,  ...,  0.3663,  1.4028,  1.4076],
          [ 0.0825, -1.6777, -1.8811,  ...,  0.2698,  1.5417, -0.2626],
          [ 0.1157, -1.9528,  0.0195,  ...,  0.7288, -0.1998,  0.4658],
          ...,
          [ 0.2175, -0.5989, -0.1006,  ...,  1.2585,  0.4564, -0.4092],
          [-0.2639,  1.6002,  0.3030,  ..., -0.1700, -0.6620,  0.0823],
          [ 0.3655, -2.0442,  0.9534,  ..., -0.4443,  0.3536,  0.5260]]]])


In [7]:
rand_output = neural_net(rand_input)
print(rand_output)

tensor([[ 0.0357,  0.0424,  0.0950, -0.0865,  0.1682, -0.0839,  0.0280, -0.0366,
         -0.0928,  0.0673]], grad_fn=<AddmmBackward>)


In [8]:
neural_net.zero_grad()
rand_output.backward(torch.randn(1, 10), retain_graph= True)

In [9]:
# Loss Function
target_output = torch.randn(10).view(1, -1)
criterion = nn.MSELoss()

loss = criterion(rand_output, target_output)
print(loss, loss.item())

tensor(0.9651, grad_fn=<MseLossBackward>) 0.9651324152946472


In [10]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [11]:
# Backprop
neural_net.zero_grad()
print('conv1.bias.grad before backward')
print(neural_net.conv_layer_one.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(neural_net.conv_layer_one.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0127,  0.0055,  0.0026,  0.0027, -0.0030, -0.0136])


In [12]:
# Update Weights
# SGD = Stochastic Gradient Descent
## W = W - LR * G

learning_rate = 0.01
for feature in neural_net.parameters():
    feature.data.sub_(learning_rate * feature.grad.data)

In [13]:
# However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

optimizer = opt.SGD(neural_net.parameters(), lr=0.01) ## used different types

### In your training loop 
optimizer.zero_grad() ## clear graidient buffers
output = neural_net(rand_input)
loss = criterion(output, target_output)
loss.backward()
optimizer.step()